In [63]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime,timedelta
import time
import talib as ta
import pytz
import warnings
warnings.filterwarnings("ignore")

In [69]:
ativo = 'WING23'
#ativo = 'EURUSD'



if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [4]:

def comprar(symbol):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.0,
        "type": mt5.ORDER_TYPE_BUY,
        "price": mt5.symbol_info_tick(symbol).ask,
        "sl": 0.0,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "Compra-5M",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    result = mt5.order_send(request)
    print(result)
    
    return 'call'

def vender(symbol):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.0,
        "type": mt5.ORDER_TYPE_SELL,
        "price": mt5.symbol_info_tick(symbol).bid,
        "sl": 0.0,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "Venda-5M",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    print(result)

    return 'sell'

def check_close(timestamp_abertura, acao = ''):
    tempo_espera = 1
    conserto_timestamp = 7200
    tempo_fechamento = timestamp_abertura + tempo_espera * 60
    agora_timestamp = int(datetime.now().timestamp()) + conserto_timestamp
    while tempo_fechamento > agora_timestamp:
        print(str(datetime.fromtimestamp(tempo_fechamento)))
        agora_timestamp = int(datetime.now().timestamp()) + conserto_timestamp
        print(str(datetime.fromtimestamp(agora_timestamp)))
        time.sleep(2)

    print('passou')

    mt5.Close(symbol = 'EURUSD')

    '''  symbol = 'EURUSD'
    if acao == 'call':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif acao =='sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid

    close_request={
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 1.0,
    "type": trade_type,
    "price": price,
    "deviation": 20,
    "magic": 234000,
    "comment": f"acao",
    "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
    "type_filling": mt5.ORDER_FILLING_FOK,
    }

    
    # send a close request
    result=mt5.order_send(close_request)

    print(result)'''

    
def verificar():
    if True:
        comprar('EURUSD')
        return 'call'
    else:
        return 'sell'
    

In [5]:
acao = ''
while True: 
    positions=mt5.positions_get()
    if positions:
        timestamp_abertura = positions[0].time
        check_close(timestamp_abertura, acao = acao)
    else:
        data_e_hora_atuais = datetime.now()
        hora_teste = str(data_e_hora_atuais.second) 
        if hora_teste == '0':
            acao = verificar()
            print('ok')
            time.sleep(1)


OrderSendResult(retcode=10009, deal=5563138, order=6568865, volume=1.0, price=1.06577, bid=0.0, ask=0.0, comment='Request executed', request_id=62, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='EURUSD', volume=1.0, price=1.06577, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='Compra-5M', position=0, position_by=0))
ok
2022-12-27 01:05:01
2022-12-27 01:04:01
2022-12-27 01:05:01
2022-12-27 01:04:03
2022-12-27 01:05:01
2022-12-27 01:04:05
2022-12-27 01:05:01
2022-12-27 01:04:07
2022-12-27 01:05:01
2022-12-27 01:04:09
2022-12-27 01:05:01
2022-12-27 01:04:11
2022-12-27 01:05:01
2022-12-27 01:04:13
2022-12-27 01:05:01
2022-12-27 01:04:15
2022-12-27 01:05:01
2022-12-27 01:04:17
2022-12-27 01:05:01
2022-12-27 01:04:19
2022-12-27 01:05:01
2022-12-27 01:04:21
2022-12-27 01:05:01
2022-12-27 01:04:23
2022-12-27 01:05:01
2022-12-27 01:04:25
2022-12-27 01:05:01
2022-12-27 01:04:27
2022-12-27 01:05:01
2022-1

KeyboardInterrupt: 

In [224]:
tempo = -5
ativo = 'WDO@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M15, 0, 5000))
df.columns = ['Data','Open','High','Low','Close','Vol','','']
time_fix = 10800
df['Data'] = df['Data'].apply(lambda x: str(datetime.fromtimestamp(x + time_fix)))
df['MA_9'] = ta.EMA(df['Close'], timeperiod =6)
df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
df['MA_9_tend_diff'] = df['MA_9_tend'].diff()
df['MA_100'] = ta.SMA(df['Close'],timeperiod=100)
df['tend'] = df.apply(lambda x: 1 if x['Close'] > x['MA_100'] else 0, axis = 1)

df['acao'] = df.apply(lambda x: 'sell' if (x['MA_9_tend_diff'] == 2 and x['tend'] == 0) 
    else 'call' if  (x['MA_9_tend_diff'] == -2 and x['tend'] == 1)  else 0, axis =1)

df['preco'] = df.apply(lambda x: x['Close'], axis =1)

df['stop'] = df.apply(lambda x: x['High'] + (x['High'] - x['Low']) * 2 if x['acao']=='sell' else x['Low'] - (x['High'] - x['Low']) * 2 if x['acao'] == 'call' else 0, axis =1)

df['realizacao_simplista'] = df['Close'].shift(tempo)

##Excluir seguidos
lista = df['acao'].values

tempo_lista = abs(tempo)
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

df['acao'] = lista




In [225]:
df

,Data,Open,High,Low,Close,Vol,,,MA_9,MA_9_diff_value,MA_9_tend,MA_9_tend_diff,MA_100,tend,acao,preco,stop,realizacao_simplista
0,2022-06-14 10:45:00,5146.0,5154.5,5135.5,5139.5,56675,500,174203,NaN,NaN,-1,NaN,NaN,0,0,5139.5,0.0,5143.5
1,2022-06-14 11:00:00,5139.5,5149.5,5133.5,5144.0,46736,500,151331,NaN,NaN,-1,0.0,NaN,0,0,5144.0,0.0,5146.0
2,2022-06-14 11:15:00,5143.5,5149.0,5140.5,5147.5,25744,500,84388,NaN,NaN,-1,0.0,NaN,0,0,5147.5,0.0,5150.0
3,2022-06-14 11:30:00,5147.0,5154.0,5137.0,5149.0,39345,500,120156,NaN,NaN,-1,0.0,NaN,0,0,5149.0,0.0,5158.0
4,2022-06-14 11:45:00,5149.5,5149.5,5134.5,5147.5,37422,500,111239,NaN,NaN,-1,0.0,NaN,0,0,5147.5,0.0,5155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-12-28 17:15:00,5258.0,5260.5,5257.0,5259.0,1287,0,13629,5255.788133,1.284747,1,0.0,5253.935,1,0,5259.0,0.0,NaN
4996,2022-12-28 17:30:00,5259.0,5264.0,5259.0,5260.5,1852,0,18100,5257.134381,1.346248,1,0.0,5254.580,1,0,5260.5,0.0,NaN
4997,2022-12-28 17:45:00,5261.0,5267.0,5260.0,5265.5,1694,0,15780,5259.524558,2.390177,1,0.0,5255.325,1,0,5265.5,0.0,NaN
4998,2022-12-28 18:00:00,5265.0,5265.5,5260.5,5264.0,1532,0,10253,5260.803255,1.278698,1,0.0,5256.040,1,0,5264.0,0.0,NaN


In [226]:
def cal_result_new(x):
    if x['acao'] == 'sell':
        if x['stop_real'] == 1:
            resultado = x['preco'] - x['stop']
            return resultado
        else:
            resultado = x['preco'] - x['realizacao_simplista']
            return resultado
    elif x['acao'] == 'call':
        if x['stop_real'] == 1:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado

for index, row in df.iterrows():
        tempo = abs(tempo)
        if row['stop'] != 0 and row['acao'] == 'call':
            for i in range(tempo):
                try:
                    if df.loc[index + i + 1,'Low'] <= row['stop']: 
                            df.loc[index,'stop_real'] = 1
                except:
                    pass
        elif row['stop'] != 0 and row['acao'] == 'sell':
            for i in range(tempo):
                try:
                    if df.loc[index + i + 1,'High'] >= row['stop']: 
                            df.loc[index,'stop_real'] = 1
                except:
                    pass

    
df = df.fillna(0)


#Deixar apenas o DataFrame quando houve ação
df_acao = df[df['acao'] != 0]



df_acao['resultado_valor'] = df_acao.apply(cal_result_new, axis = 1)
df_acao['resultado_binario'] = df_acao['resultado_valor'].apply(lambda x: 1 if x > 0 else 0)


compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: 0.46919431279620855% -- Compra: 0.5235849056603774%
Venda: -149.0 -- Compra: 410.0
